In [25]:
%cd ..

/home/soda/rcappuzz/work


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import pickle
import polars as pl

In [28]:
import src.table_integration.join_profiling as jp

In [29]:
with open("generated_candidates.pickle", "rb") as fp:
    generated_candidates = pickle.load(fp)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_candidates.pickle'

In [30]:
one_candidate = generated_candidates["minhash"].popitem()

In [31]:
hash_, mdata = one_candidate

In [32]:
source_metadata = mdata.source_metadata
candidate_metadata = mdata.candidate_metadata

In [33]:
source_table = pl.read_parquet(source_metadata["full_path"])
candidate_table = pl.read_parquet(candidate_metadata["full_path"])

In [79]:
left_on = mdata.left_on
right_on = mdata.right_on


In [98]:
unique_source = jp.find_unique_keys(source_table, left_on)
unique_cand = jp.find_unique_keys(candidate_table, right_on)

In [99]:
jp.measure_cardinality_proportion(source_table, candidate_table)

2.962121212121212

In [82]:
unique_source = source_table[left_on].unique()
unique_cand = candidate_table[right_on].unique()
intersection=source_table.join(unique_cand, left_on=left_on, right_on=right_on, how="inner")
containment = len(intersection)/len(source_table)
print(containment)

0.3375959079283887


In [83]:
jp.measure_join_quality(source_table, candidate_table, left_on, right_on)

2

In [84]:
%%timeit
jp.measure_containment(unique_source, unique_cand, left_on=left_on, right_on=right_on)

913 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [85]:
%%timeit
s1 = set(unique_source.to_series())
s2 = set(unique_cand.to_series())
len(s1.intersection(s2))/len(unique_source)

395 µs ± 52.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [76]:
on = ["subject", "isLocatedIn"]
left_on = right_on = on
source_table = source_table[left_on].unique()
unique_cand = candidate_table[right_on].unique()
# jp.measure_containment(unique_source, unique_cand, on, on)

In [77]:
%%timeit
s1 = set(unique_source.to_series())
s2 = set(unique_cand.to_series())
len(s1.intersection(s2))/len(unique_source)

39.3 ms ± 3.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [78]:
%%timeit
jp.measure_containment(unique_source, unique_cand, left_on=left_on, right_on=right_on)

2.72 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
one_candidate = generated_candidates["minhash"].popitem()


In [41]:
l_pd = source_table[left_on].drop_nulls().to_pandas()
r_pd = candidate_table[right_on].drop_nulls().to_pandas()

In [45]:
len(l_pd) * len(r_pd) / len(source_table)

745496.2941176471

In [50]:
len(l_pd) * len(r_pd)

291489051